NOMBRES: Fátima Priscilla 

APELLIDOS: González Sandoval

CARNE: 20689

FECHA: 24 de agosto de 2024

**Ejercicio 1**
Con los datos de la clase de lunes, cálcule PPMI, pero aplicando Lapace Smoothing.

**Ejercicio 2**

POC para crear información de entreno

In [1]:
#Librerías que necesitarán
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
%load_ext tensorboard
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [2]:
sentece = "The wind Crosses the brown land unheard" #De mis poemas fav. The waste land de T.S Elliot
tokens = tf.keras.preprocessing.text.text_to_word_sequence(sentece) #tokenizar
vocab, index = {}, 1 #arreglo vocabulario, empezar indice en 1
vocab["algo importante para que no explote el programa"] = 0

#Llene el arreglo para vocabulario
for word in tokens:
    if word not in vocab:
        vocab[word] = index
        index += 1

vocab_size = len(vocab)
example_sequence = [vocab[word] for word in tokens]
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
    example_sequence,
    vocabulary_size=vocab_size,
    window_size=window_size,
    negative_samples=0
) #rellene esta area como le sea pertinente)

print("Tokens:", tokens)
print("Vocabulario:", vocab)
print("Secuencia de ejemplo:", example_sequence)
print("Positive Skip-Grams:", positive_skip_grams)

Tokens: ['the', 'wind', 'crosses', 'the', 'brown', 'land', 'unheard']
Vocabulario: {'algo importante para que no explote el programa': 0, 'the': 1, 'wind': 2, 'crosses': 3, 'brown': 4, 'land': 5, 'unheard': 6}
Secuencia de ejemplo: [1, 2, 3, 1, 4, 5, 6]
Positive Skip-Grams: [[1, 4], [3, 1], [3, 4], [1, 2], [5, 1], [1, 3], [5, 4], [2, 1], [4, 5], [6, 5], [4, 6], [2, 3], [4, 1], [2, 1], [6, 4], [1, 2], [1, 5], [3, 2], [5, 6], [3, 1], [4, 3], [1, 3]]


In [3]:
#Resultados
inverse_vocab = {index: token for token, index in vocab.items()}
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(1, 4): (the, brown)
(3, 1): (crosses, the)
(3, 4): (crosses, brown)
(1, 2): (the, wind)
(5, 1): (land, the)


In [4]:
target_word, context_word = positive_skip_grams[0]
# Escoja un numero para muestras negativos (que no pertenence al contexto)
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # clase que debe ser muestreada como positiva (que pertenece al contexto)
    num_true=1,
    num_sampled=num_ns,
    unique=True,
    range_max=vocab_size,  # [0, vocab_size]
    seed=SEED,
    name="negative_sampling"
)

In [5]:
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['wind', 'the', 'brown', 'crosses']


In [6]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=vocab_size) #funcion que construye la tabla de muestreo en forma de frecuencias
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381]


In [7]:
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1) #se agrega una dimension para poder concatenar
context = tf.concat([context_class, negative_sampling_candidates], 0)
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

#Listo! asi se prepara la info para entrenar

In [8]:
#Informacion a utilizar:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
#Paso 1: cree una funcion que estandarice el texto como ya hemos hecho e incrustela en una capa de Tensorflow (TextVectorization)





In [ ]:
vectorize_layer.adapt(text_ds.batch(1024))

In [ ]:
#Paso 2: Usando la POC de creacion de datos de entreno, defina una funcion para este proceso






In [ ]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()
sequences = list(text_vector_ds.as_numpy_iterator())

In [ ]:
#El siguiente codigo es el modelo Word2Vec usando lo que ya han hecho. Sin embargo, deben agregar la metrica de similitud que vimos en clase, a la cual deben llamar dots
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)


    # dots: (batch, context)
    return dots

In [ ]:
#Entrenar modelo y definir CategoricalCrossEntropy como funcion de perdida. Mostrar el print de accuracy y loss. Si deciden presentar sus resultados en tensorboard, adjuntar capturas.